In [2]:
特征：
   'label','popularity','carCommentVolum','newsReplyVolum'平移了12个月
    train:
        1,  2,  3,  4,  5,  6,  7,  8,  9,  10,  11,  12
        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,  23,  24 
        
    test: 21,22,23,24

SyntaxError: invalid character in identifier (<ipython-input-2-027fb5b461bb>, line 1)

In [18]:
# import sys
import numpy as np
import pandas as pd
# import os 
# import gc
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
import datetime
import time
import lightgbm as lgb
import xgboost as xgb
import catboost as ctb
import warnings
from sklearn.cluster import KMeans
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
ctb_params = {
'n_estimators': 6000,
'learning_rate': 0.02,
'random_seed': 2019,
'reg_lambda': 0.08,
'subsample': 0.7,
'bootstrap_type': 'Bernoulli',
'boosting_type': 'Plain',
'one_hot_max_size': 10,
'rsm': 0.5,
'leaf_estimation_iterations': 5,
'use_best_model': True,
'max_depth': 6,
'verbose': -1,
'thread_count': 4
}

In [19]:
# path  = './ccf_car/'
train_sales  = pd.read_csv('train_sales_data.csv')


train_search = pd.read_csv('train_search_data.csv')
train_user   = pd.read_csv('train_user_reply_data.csv')
evaluation_public = pd.read_csv('evaluation_public.csv')
# submit_example    = pd.read_csv('submit_example.csv')

data = pd.concat([train_sales, evaluation_public], ignore_index=True)
data = data.merge(train_search, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user, 'left', on=['model', 'regYear', 'regMonth'])
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])

#索引-->value
#LabelEncoder
for i in ['bodyType', 'model']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))
    #key-->value
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']
data['happyNY'] = 0
data.loc[(data['regYear'].isin([2016,2018])&data['regMonth'].isin([2])),'happyNY'] = 1
data.loc[(data['regYear'].isin([2017])&data['regMonth'].isin([1])),'happyNY'] = 1
a = 6; b = 4
data['weightMonth'] = data['regMonth'].map({1:a, 2:a, 3:a, 4:a,
                                            5:b, 6:b, 7:b, 8:b, 9:b, 10:b, 11:b, 12:b,})

initial_key_list = ['weightMonth','happyNY','adcode','bodyType','forecastVolum','id','model','province','regMonth','regYear','salesVolume','popularity','carCommentVolum','newsReplyVolum'	,'label','mt']
data = data[initial_key_list].copy()
data['pred_label'] = 0

data.tail()

,weightMonth,happyNY,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum,label,mt,pred_label
36955,6,0,350000,0,NaN,5364,59,福建,4,2018,NaN,NaN,NaN,NaN,NaN,28,0
36956,6,0,210000,0,NaN,5365,59,辽宁,4,2018,NaN,NaN,NaN,NaN,NaN,28,0
36957,6,0,500000,0,NaN,5366,59,重庆,4,2018,NaN,NaN,NaN,NaN,NaN,28,0
36958,6,0,610000,0,NaN,5367,59,陕西,4,2018,NaN,NaN,NaN,NaN,NaN,28,0
36959,6,0,230000,0,NaN,5368,59,黑龙江,4,2018,NaN,NaN,NaN,NaN,NaN,28,0


In [20]:
def Cluster(df_,pred_month): 
    df_['cluster_label']  =-1
    df = df_[df_['mt']<pred_month]
    df= df[['adcode','model','mt','label']].set_index(['model','adcode','mt']).reset_index()
    kmeans = KMeans(n_clusters = 3) 
    kmeans.fit(df.values)
    df_.loc[df_['mt']<pred_month,'cluster_label'] =  kmeans.labels_
#     print(df.head())
#     df = df.merge(df,'left',on = ['adcode','model','mt','label'])
    return df_




In [22]:
def windows_feature(data):
#     if (judge >1):
    print("windows_var_features start")
    df = data.copy()
    df_all = pd.DataFrame()
    df['model_adcode'] = df['adcode'] + df['model']
    model_adcode = list(df.model_adcode.unique())
    for i in tqdm(model_adcode):
        idx = (df['model_adcode']==i)
        df2 = df[idx] 

        df2['windows3_var'] = df2['label'].rolling(3).var()
        df2['windows5_var'] = df2['label'].rolling(5).var()
        df2['windows7_var'] = df2['label'].rolling(7).var()

        df2['windows3_var'] = df2['windows3_var'].rolling(3).var()
        df2['windows5_var'] = df2['windows5_var'].rolling(5).var()
        df2['windows7_var'] = df2['windows7_var'].rolling(7).var()
        

        df_all = pd.concat([df_all, df2], ignore_index=True)
    data = data.merge(df_all, 'left', on=['weightMonth','happyNY','cluster_label','adcode','bodyType','forecastVolum','id','model','province','regMonth','regYear','salesVolume','popularity','carCommentVolum','newsReplyVolum','pred_label','label','mt'])
    print("windows_var_features end")
    return data

        

In [25]:
def get_stat_feature(df_):   
    df = df_.copy()
    stat_feat = []
    df['model_adcode'] = df['adcode'] + df['model']
    df['model_adcode_mt'] = df['model_adcode'] * 100 + df['mt']
    for col in tqdm(['label']):#,,'popularity','newsReplyVolum'
    # shift
        for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
            stat_feat.append('shift_model_adcode_mt_{}_{}'.format(col,i))
            df['model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'] + i
            df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,i))
            df['shift_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])

#     for i in [1,2,3]: #本月销量和前1-3,12个月相减
#         df["differ_{}_{}".format(col,i)] = df['{}'.format(col)] -df['shift_model_adcode_mt_{}_{}'.format(col,i)]
#         df["differ2_{}_{}".format(col,i)] = df["differ_{}_{}".format(col,i)]-df.groupby('model_adcode')["differ_{}_{}".format(col,i)].shift(1)
#         df["differ3_{}_{}".format(col,i)] = df["differ2_{}_{}".format(col,i)]-df.groupby('model_adcode')["differ2_{}_{}".format(col,i)].shift(1)
#         stat_feat.append("differ3_{}_{}".format(col,i))

#             df["ratio_{}_{}".format(col,i)] = df['{}'.format(col)] /df['shift_model_adcode_mt_{}_{}'.format(col,i)]
#             df["ratio2_{}_{}".format(col,i)] = df["ratio_{}_{}".format(col,i)]/df.groupby('model_adcode')["ratio_{}_{}".format(col,i)].shift(1)
#             stat_feat.append("ratio2_{}_{}".format(col,i))
    return df,stat_feat

In [26]:
def get_stat_feature2(df_):  #bodytype+省份
    df = df_.copy()
    print(df.shape)
    stat2_feat = []
    df['body_adcode'] = df['adcode'] + df['bodyType']
    df['body_adcode_mt'] = df['body_adcode'] * 100 + df['mt']
    df2 = df.copy()
    for col in tqdm(['label','popularity','newsReplyVolum']):
        df = df.groupby(['bodyType','adcode','body_adcode','mt','body_adcode_mt'])[col].mean().reset_index()
        for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
            df['shift_{}_{}_body_adcode'.format(col,i)] = df.groupby('body_adcode')[col].shift(i)
            stat2_feat.append('shift_{}_{}_body_adcode'.format(col,i))
        df2 = df2.merge(df,'left',on = ['bodyType','adcode','mt','body_adcode_mt','body_adcode',col])
        df = df2.copy()
    
    return df2,stat2_feat

In [27]:
def get_stat_feature3(df_):  #省份
    df = df_.copy()
    print(df.shape)
    stat3_feat = []
#     df['adcode'] = df['adcode'] + df['bodyType']
    df['adcode_mt'] = df['adcode'] * 100 + df['mt']
    df2 = df.copy()
    for col in tqdm(['label','popularity','newsReplyVolum']):
        df = df.groupby(['adcode','mt','adcode_mt'])[col].mean().reset_index()
        for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
            df['shift_{}_{}_adcode'.format(col,i)] = df.groupby('adcode')[col].shift(i)
            stat3_feat.append('shift_{}_{}_adcode'.format(col,i))
        df2 = df2.merge(df,'left',on = ['adcode','mt','adcode_mt',col])
        df = df2.copy()
    return df2,stat3_feat


In [28]:
def valid_score(data, pred='pred_label', label='label', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

In [29]:
def online_score(data, pred='pred_label', label='true_sales', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    y = pd.read_csv("valid_data_y.csv")
    data = data.merge(y, 'left', on=['id'])
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

In [30]:
def get_model_type(train_x,train_y,valid_x,valid_y,m_type='lgb'):   
    if m_type == 'lgb':
        model = lgb.LGBMRegressor(
                                num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
                                max_depth=-1, learning_rate=0.01, min_child_samples=1, random_state=1024,
                                n_estimators=2000, subsample=0.9, colsample_bytree=0.7,
                                )
        model.fit(train_x, train_y, 
                  
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              categorical_feature=cate_feat, 
              early_stopping_rounds=100, verbose=100)      
    elif m_type == 'xgb':
        model = xgb.XGBRegressor(
                                max_depth=5 , learning_rate=0.01, n_estimators=4000, 
                                objective='reg:gamma', tree_method = 'hist',subsample=0.9, 
                                colsample_bytree=0.7, min_child_samples=5,eval_metric = 'rmse' 
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              early_stopping_rounds=100, verbose=100)
    elif m_type == 'ctb':
        model = ctb.CatBoostRegressor(**ctb_params, cat_features=cate_feat, early_stopping_rounds=100)

        model.fit(train_x, train_y,
           eval_set=[(valid_x, valid_y)],
                      #eval_metric='mae',
                      verbose=100)
    return model

In [31]:
def get_train_model(df_, m, m_type):
    df = df_.copy()
    # 数据集划分
    st = 9
    all_idx   = (df['mt'].between(st , m-1))
    train_idx = (df['mt'].between(st , m-5))
    valid_idx = (df['mt'].between(m-4, m-4))
    test_idx  = (df['mt'].between(m  , m  ))
    
    print('all_idx  :',st ,m-1)
    print('train_idx:',st ,m-5)
    print('valid_idx:',m-4,m-4)
    print('test_idx :',m  ,m  )  

    # 最终确认
    train_x = df[train_idx][features]
    train_y = df[train_idx]['label']
    valid_x = df[valid_idx][features]
    valid_y = df[valid_idx]['label']   
    # get model
    model = get_model_type(train_x,train_y,valid_x,valid_y,m_type)  
    # offline
    df['pred_label'] = model.predict(df[features])
    best_score = valid_score(df[valid_idx]) 
    # online
    if m_type == 'lgb':
        model.n_estimators = model.best_iteration_ + 100
        model.fit(df[all_idx][features], df[all_idx]['label'], categorical_feature=cate_feat)
    elif m_type == 'xgb':
        model.n_estimators = model.best_iteration + 100
        model.fit(df[all_idx][features], df[all_idx]['label'])
#     elif m_type == 'ctb':
# #         model.n_estimators = model.best_iteration_ + 100
#         model.fit(df[all_idx][features], df[all_idx]['label'], cat_features=cate_feat)
    df['forecastVolum'] =  model.predict(df[features])
#     print('valid mean:',df[valid_idx]['pred_label'].mean())
#     print('true  mean:',df[valid_idx]['label'].mean())
#     print('test  mean:',df[test_idx]['forecastVolum'].mean())
    print("--------------------------")
    # 阶段结果
    sub = df[test_idx][['id']]
    sub['forecastVolum'] = df[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    print(sub.shape)
    return sub,df[valid_idx]['pred_label']

In [32]:
def sanlie(data):
    # col, col2, col3 中 ，设1.5倍四分位距之外的数据为异常值，用上下四分位数的均值填充
    df = data.copy()
    col, col2, col3 = ['popularity', 'carCommentVolum', 'newsReplyVolum']#, 'carCommentVolum', 'newsReplyVolum'
    col_per = np.percentile(df[col],(25,75))
    diff = 1.5*(col_per[1] - col_per[0])
    col_per_in = (df[col] >= col_per[0] - diff) & (df[col] <= col_per[1] + diff) 

    col_per2 = np.percentile(df[col2],(25,75))
    diff2 = 1.5*(col_per2[1] - col_per2[0])
    col_per_in2 = (df[col2] >= col_per2[0] - diff2) & (df[col2] <= col_per2[1] + diff2)

    col_per3 = np.percentile(df[col3],(25,75))
    diff3 = 1.5*(col_per3[1] - col_per3[0])
    col_per_in3 = (df[col3] >= col_per3[0] - diff3) & (df[col3] <= col_per3[1] + diff3)

    df.loc[~col_per_in, col] = col_per.mean()
    df.loc[~col_per_in2, col2] = col_per2.mean()
    df.loc[~col_per_in3, col3] = col_per3.mean()

    # 统计销量
    df['bt_ry_mean'] = df.groupby(['bodyType','regYear'])['salesVolume'].transform('mean')
    df['ad_ry_mean'] = df.groupby(['adcode','regYear'])['salesVolume'].transform('mean')
    df['md_ry_mean'] = df.groupby(['model','regYear'])['salesVolume'].transform('mean')
    return df

In [33]:
for month in [25,26,27,28]: 
#     judge = month-20
    m_type = 'xgb'
    data = Cluster(data,month)
    data_df = windows_feature(data)
#     data_df = data.copy()
    data_df2, stat_feat = get_stat_feature(data_df)
    data_df3, stat2_feat = get_stat_feature2(data_df2)
    data_df4, stat3_feat = get_stat_feature3(data_df3)


    win_feat = ['windows3_var','windows5_var','windows7_var']
#     win_feat = []
#     winPop_feat = ['windowsPop4_var','windowsPop4_std','windowsPop5_var','windowsPop5_std','windowsPop12_var','windowsPop12_std']
#     ,'windows6_var','windows6_std']
#     qushi_feat = ['ratio2','differ2']
    num_feat = ['regYear','regMonth','weightMonth'] + stat_feat+stat2_feat+stat3_feat +win_feat#,'windows_var','windows_std'
    cate_feat = ['adcode','bodyType','model','model_adcode','body_adcode','cluster_label','happyNY']

    if m_type == 'lgb':
        for i in cate_feat:
            data_df4[i] = data_df4[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df4[i] = lbl.fit_transform(data_df4[i].astype(str))
        

    features = num_feat + cate_feat
    print("xxxxxxxxxxxxxxxxxx")
    print(len(features), len(set(features)))
    sub,val_pred = get_train_model(data_df4, month, m_type)
#     online_test_idx  = (data_df4['mt'].between(21  , 24  ))
    print("---")
    print(data.loc[(data.regMonth==month)].shape)
    data.loc[(data.mt==month), 'salesVolume'   ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'label'      ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'pred_label'      ] = sub['forecastVolum'].values
#     data['n_label'] = data['label'] / data['model_weight']

    # # sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
# # sub.columns = ['id','forecastVolum']
# # sub[['id','forecastVolum']].round().astype(int).to_csv('submit/yulao_lgb.csv', index=False)
# best_score = online_score(data[online_test_idx]) 

windows_var_features start


100%|██████████████████████████████████████████████████████████████████████████████| 1320/1320 [00:19<00:00, 67.60it/s]


windows_var_features end


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.88it/s]


(36960, 47)


...

(36960, 47)


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.40it/s]


(36960, 85)


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 33.03it/s]


xxxxxxxxxxxxxxxxxx
97 97
all_idx  : 9 27
train_idx: 9 23
valid_idx: 24 24
test_idx : 28 28
[0]	validation_0-rmse:995.968	validation_1-rmse:1451.31
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:995.439	validation_1-rmse:1450.78
[200]	validation_0-rmse:994.013	validation_1-rmse:1449.33
[300]	validation_0-rmse:990.205	validation_1-rmse:1445.44
[400]	validation_0-rmse:980.3	validation_1-rmse:1435.26
[500]	validation_0-rmse:955.975	validation_1-rmse:1409.78
[600]	validation_0-rmse:902.263	validation_1-rmse:1351.72
[700]	validation_0-rmse:801.836	validation_1-rmse:1237.92
[800]	validation_0-rmse:651.751	validation_1-rmse:1057.96
[900]	validation_0-rmse:479.392	validation_1-rmse:840.922
[1000]	validation_0-rmse:328.467	validation_1-rmse:642.349
[1100]	validation_0-rmse:227.44	validation_1-rmse:503.804
[1200]	validation_0-rmse:176.795	validation_1-rmse:425.5

In [ ]:
0.6207756606359476
661
536
610
637
6399
6537

xxxxxxxxxxxxxxxxxx
94 94
all_idx  : 9 24
train_idx: 9 20
valid_idx: 21 21
test_idx : 25 25
[0]	validation_0-rmse:5.44972	validation_1-rmse:5.53741
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:4.77321	validation_1-rmse:4.86134
[200]	validation_0-rmse:3.58254	validation_1-rmse:3.66603
[300]	validation_0-rmse:2.15474	validation_1-rmse:2.22622
[400]	validation_0-rmse:1.06133	validation_1-rmse:1.11433
[500]	validation_0-rmse:0.497894	validation_1-rmse:0.534733
[600]	validation_0-rmse:0.294178	validation_1-rmse:0.318593
[700]	validation_0-rmse:0.240846	validation_1-rmse:0.256605
[800]	validation_0-rmse:0.225619	validation_1-rmse:0.242352
[900]	validation_0-rmse:0.217996	validation_1-rmse:0.239598
[1000]	validation_0-rmse:0.212463	validation_1-rmse:0.23944
Stopping. Best iteration:
[937]	validation_0-rmse:0.215771	validation_1-rmse:0.239281

0.7866598787835365
--------------------------
(1320, 2)
---
(0, 18)
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.81it/s]
(36960, 44)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.81it/s]
(36960, 82)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  7.14it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 30.92it/s]
xxxxxxxxxxxxxxxxxx
94 94
all_idx  : 9 25
train_idx: 9 21
valid_idx: 22 22
test_idx : 26 26
[0]	validation_0-rmse:5.45651	validation_1-rmse:5.46999
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:4.77983	validation_1-rmse:4.79313
[200]	validation_0-rmse:3.58826	validation_1-rmse:3.5934
[300]	validation_0-rmse:2.15822	validation_1-rmse:2.14294
[400]	validation_0-rmse:1.06247	validation_1-rmse:1.02567
[500]	validation_0-rmse:0.497371	validation_1-rmse:0.466585
[600]	validation_0-rmse:0.292633	validation_1-rmse:0.296461
[700]	validation_0-rmse:0.238894	validation_1-rmse:0.271606
[800]	validation_0-rmse:0.223627	validation_1-rmse:0.273205
Stopping. Best iteration:
[722]	validation_0-rmse:0.234248	validation_1-rmse:0.271113

0.7451483930356577
--------------------------
(1320, 2)
---
(0, 18)
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.34it/s]
(36960, 44)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  7.60it/s]
(36960, 82)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.85it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 28.22it/s]
xxxxxxxxxxxxxxxxxx
94 94
all_idx  : 9 26
train_idx: 9 22
valid_idx: 23 23
test_idx : 27 27
[0]	validation_0-rmse:5.45748	validation_1-rmse:5.57606
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:4.78083	validation_1-rmse:4.89818
[200]	validation_0-rmse:3.58916	validation_1-rmse:3.69693
[300]	validation_0-rmse:2.15882	validation_1-rmse:2.24187
[400]	validation_0-rmse:1.06268	validation_1-rmse:1.11094
[500]	validation_0-rmse:0.498201	validation_1-rmse:0.501153
[600]	validation_0-rmse:0.294335	validation_1-rmse:0.265962
[700]	validation_0-rmse:0.242073	validation_1-rmse:0.209591
[800]	validation_0-rmse:0.227329	validation_1-rmse:0.202261
[900]	validation_0-rmse:0.219838	validation_1-rmse:0.201757
Stopping. Best iteration:
[880]	validation_0-rmse:0.220823	validation_1-rmse:0.2015

0.7938835552300731
--------------------------
(1320, 2)
---
(0, 18)
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.28it/s]
(36960, 44)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  7.32it/s]
(36960, 82)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  5.53it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 23.18it/s]
xxxxxxxxxxxxxxxxxx
94 94
all_idx  : 9 27
train_idx: 9 23
valid_idx: 24 24
test_idx : 28 28
[0]	validation_0-rmse:5.46546	validation_1-rmse:5.82129
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:4.78855	validation_1-rmse:5.14099
[200]	validation_0-rmse:3.59583	validation_1-rmse:3.93229
[300]	validation_0-rmse:2.16297	validation_1-rmse:2.46133
[400]	validation_0-rmse:1.06414	validation_1-rmse:1.299
[500]	validation_0-rmse:0.497369	validation_1-rmse:0.670774
[600]	validation_0-rmse:0.292647	validation_1-rmse:0.42723
[700]	validation_0-rmse:0.24013	validation_1-rmse:0.35898
[800]	validation_0-rmse:0.2252	validation_1-rmse:0.338391
[900]	validation_0-rmse:0.21829	validation_1-rmse:0.331817
[1000]	validation_0-rmse:0.213271	validation_1-rmse:0.328616
[1100]	validation_0-rmse:0.209447	validation_1-rmse:0.327698
[1200]	validation_0-rmse:0.206204	validation_1-rmse:0.326882
[1300]	validation_0-rmse:0.202961	validation_1-rmse:0.325324
[1400]	validation_0-rmse:0.200031	validation_1-rmse:0.324815
[1500]	validation_0-rmse:0.197513	validation_1-rmse:0.324421
[1600]	validation_0-rmse:0.195259	validation_1-rmse:0.324202
[1700]	validation_0-rmse:0.192661	validation_1-rmse:0.32347
[1800]	validation_0-rmse:0.190321	validation_1-rmse:0.322835
[1900]	validation_0-rmse:0.18837	validation_1-rmse:0.322511
[2000]	validation_0-rmse:0.186467	validation_1-rmse:0.32247
[2100]	validation_0-rmse:0.184632	validation_1-rmse:0.321975
[2200]	validation_0-rmse:0.182701	validation_1-rmse:0.321897
[2300]	validation_0-rmse:0.1808	validation_1-rmse:0.321784
[2400]	validation_0-rmse:0.17914	validation_1-rmse:0.321557
[2500]	validation_0-rmse:0.177358	validation_1-rmse:0.32125
[2600]	validation_0-rmse:0.17581	validation_1-rmse:0.320934
[2700]	validation_0-rmse:0.174006	validation_1-rmse:0.320715
Stopping. Best iteration:
[2698]	validation_0-rmse:0.174029	validation_1-rmse:0.320699

0.6900885094082845
--------------------------
(1320, 2)
---
(0, 18)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.48it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 29.61it/s]
xxxxxxxxxxxxxxxxxx
97 97
all_idx  : 9 24
train_idx: 9 20
valid_idx: 21 21
test_idx : 25 25
[0]	validation_0-rmse:984.025	validation_1-rmse:1046.81
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:983.499	validation_1-rmse:1046.28
[200]	validation_0-rmse:982.078	validation_1-rmse:1044.83
[300]	validation_0-rmse:978.285	validation_1-rmse:1040.96
[400]	validation_0-rmse:968.429	validation_1-rmse:1030.89
[500]	validation_0-rmse:944.265	validation_1-rmse:1006.07
[600]	validation_0-rmse:891.068	validation_1-rmse:950.968
[700]	validation_0-rmse:792.051	validation_1-rmse:846.827
[800]	validation_0-rmse:644.828	validation_1-rmse:689.596
[900]	validation_0-rmse:476.371	validation_1-rmse:509.325
[1000]	validation_0-rmse:329.021	validation_1-rmse:350.307
[1100]	validation_0-rmse:228.961	validation_1-rmse:238.742
[1200]	validation_0-rmse:179.757	validation_1-rmse:182.895
[1300]	validation_0-rmse:161.159	validation_1-rmse:164.735
[1400]	validation_0-rmse:151.269	validation_1-rmse:157.334
[1500]	validation_0-rmse:144.322	validation_1-rmse:152.802
[1600]	validation_0-rmse:139.295	validation_1-rmse:150.314
[1700]	validation_0-rmse:135.332	validation_1-rmse:147.474
[1800]	validation_0-rmse:132.257	validation_1-rmse:146.141
[1900]	validation_0-rmse:129.883	validation_1-rmse:144.797
[2000]	validation_0-rmse:127.592	validation_1-rmse:143.963
[2100]	validation_0-rmse:125.565	validation_1-rmse:143.071
[2200]	validation_0-rmse:123.507	validation_1-rmse:142.13
[2300]	validation_0-rmse:121.975	validation_1-rmse:141.649
[2400]	validation_0-rmse:120.315	validation_1-rmse:141.088
[2500]	validation_0-rmse:117.93	validation_1-rmse:140.421
[2600]	validation_0-rmse:116.258	validation_1-rmse:140.065
[2700]	validation_0-rmse:114.662	validation_1-rmse:139.547
[2800]	validation_0-rmse:113.002	validation_1-rmse:139.534
[2900]	validation_0-rmse:111.441	validation_1-rmse:139.56
Stopping. Best iteration:
[2818]	validation_0-rmse:112.666	validation_1-rmse:139.397

0.8133404714966406
--------------------------
(1320, 2)
---
(0, 18)
windows_var_features start
100%|██████████████████████████████████████████████████████████████████████████████| 1320/1320 [00:20<00:00, 63.79it/s]
windows_var_features end
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.57it/s]
(36960, 47)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  7.94it/s]
(36960, 85)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  5.89it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 27.85it/s]
xxxxxxxxxxxxxxxxxx
97 97
all_idx  : 9 25
train_idx: 9 21
valid_idx: 22 22
test_idx : 26 26
[0]	validation_0-rmse:988.997	validation_1-rmse:1007.38
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:988.469	validation_1-rmse:1006.85
[200]	validation_0-rmse:987.047	validation_1-rmse:1005.42
[300]	validation_0-rmse:983.248	validation_1-rmse:1001.6
[400]	validation_0-rmse:973.372	validation_1-rmse:991.634
[500]	validation_0-rmse:949.146	validation_1-rmse:967.045
[600]	validation_0-rmse:895.751	validation_1-rmse:912.188
[700]	validation_0-rmse:796.111	validation_1-rmse:807.932
[800]	validation_0-rmse:647.647	validation_1-rmse:651.355
[900]	validation_0-rmse:477.644	validation_1-rmse:472.713
[1000]	validation_0-rmse:328.249	validation_1-rmse:324.062
[1100]	validation_0-rmse:227.627	validation_1-rmse:233.14
[1200]	validation_0-rmse:178.441	validation_1-rmse:196.557
[1300]	validation_0-rmse:159.338	validation_1-rmse:188.959
[1400]	validation_0-rmse:149.744	validation_1-rmse:186.535
[1500]	validation_0-rmse:143.624	validation_1-rmse:184.869
[1600]	validation_0-rmse:139.243	validation_1-rmse:184.13
[1700]	validation_0-rmse:135.485	validation_1-rmse:183.425
[1800]	validation_0-rmse:132.906	validation_1-rmse:183.116
[1900]	validation_0-rmse:130.496	validation_1-rmse:183.289
Stopping. Best iteration:
[1828]	validation_0-rmse:132.272	validation_1-rmse:183.055

0.753802498212419
--------------------------
(1320, 2)
---
(0, 18)
windows_var_features start
100%|██████████████████████████████████████████████████████████████████████████████| 1320/1320 [00:20<00:00, 64.11it/s]
windows_var_features end
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.54it/s]
(36960, 47)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.74it/s]
(36960, 85)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.74it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 31.61it/s]
xxxxxxxxxxxxxxxxxx
97 97
all_idx  : 9 26
train_idx: 9 22
valid_idx: 23 23
test_idx : 27 27
[0]	validation_0-rmse:990.321	validation_1-rmse:1071.9
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:989.794	validation_1-rmse:1071.36
[200]	validation_0-rmse:988.371	validation_1-rmse:1069.89
[300]	validation_0-rmse:984.571	validation_1-rmse:1065.97
[400]	validation_0-rmse:974.691	validation_1-rmse:1055.73
[500]	validation_0-rmse:950.446	validation_1-rmse:1030.4
[600]	validation_0-rmse:896.997	validation_1-rmse:973.62
[700]	validation_0-rmse:797.204	validation_1-rmse:864.888
[800]	validation_0-rmse:648.4	validation_1-rmse:697.157
[900]	validation_0-rmse:477.946	validation_1-rmse:503.393
[1000]	validation_0-rmse:328.28	validation_1-rmse:338.12
[1100]	validation_0-rmse:227.952	validation_1-rmse:233.174
[1200]	validation_0-rmse:179.192	validation_1-rmse:185.181
[1300]	validation_0-rmse:162.393	validation_1-rmse:170.178
[1400]	validation_0-rmse:153.085	validation_1-rmse:163.063
[1500]	validation_0-rmse:146.08	validation_1-rmse:157.552
[1600]	validation_0-rmse:141.102	validation_1-rmse:154.106
[1700]	validation_0-rmse:136.363	validation_1-rmse:150.599
[1800]	validation_0-rmse:133.675	validation_1-rmse:148.42
[1900]	validation_0-rmse:131.344	validation_1-rmse:147.103
[2000]	validation_0-rmse:129.086	validation_1-rmse:145.67
[2100]	validation_0-rmse:127.376	validation_1-rmse:145.335
[2200]	validation_0-rmse:125.668	validation_1-rmse:144.666
[2300]	validation_0-rmse:124.055	validation_1-rmse:144.013
[2400]	validation_0-rmse:121.991	validation_1-rmse:143.303
[2500]	validation_0-rmse:120.383	validation_1-rmse:142.762
[2600]	validation_0-rmse:118.887	validation_1-rmse:142.352
[2700]	validation_0-rmse:117.121	validation_1-rmse:141.864
[2800]	validation_0-rmse:115.809	validation_1-rmse:141.763
[2900]	validation_0-rmse:114.385	validation_1-rmse:141.538
[3000]	validation_0-rmse:112.668	validation_1-rmse:141.321
[3100]	validation_0-rmse:111.417	validation_1-rmse:141.201
[3200]	validation_0-rmse:110.294	validation_1-rmse:141.172
[3300]	validation_0-rmse:108.981	validation_1-rmse:141.096
[3400]	validation_0-rmse:107.662	validation_1-rmse:140.807
[3500]	validation_0-rmse:106.411	validation_1-rmse:140.658
Stopping. Best iteration:
[3479]	validation_0-rmse:106.712	validation_1-rmse:140.625

0.8129950677543025
--------------------------
(1320, 2)
---
(0, 18)
windows_var_features start
100%|██████████████████████████████████████████████████████████████████████████████| 1320/1320 [00:20<00:00, 64.72it/s]
windows_var_features end
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.51it/s]
(36960, 47)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.26it/s]
(36960, 85)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.35it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 30.12it/s]
xxxxxxxxxxxxxxxxxx
97 97
all_idx  : 9 27
train_idx: 9 23
valid_idx: 24 24
test_idx : 28 28
[0]	validation_0-rmse:995.968	validation_1-rmse:1451.31
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:995.439	validation_1-rmse:1450.78
[200]	validation_0-rmse:994.013	validation_1-rmse:1449.33
[300]	validation_0-rmse:990.205	validation_1-rmse:1445.44
[400]	validation_0-rmse:980.3	validation_1-rmse:1435.26
[500]	validation_0-rmse:955.975	validation_1-rmse:1409.78
[600]	validation_0-rmse:902.263	validation_1-rmse:1351.72
[700]	validation_0-rmse:801.836	validation_1-rmse:1237.92
[800]	validation_0-rmse:651.751	validation_1-rmse:1057.96
[900]	validation_0-rmse:479.392	validation_1-rmse:840.922
[1000]	validation_0-rmse:328.466	validation_1-rmse:642.352
[1100]	validation_0-rmse:227.551	validation_1-rmse:505.478
[1200]	validation_0-rmse:176.503	validation_1-rmse:427.081
[1300]	validation_0-rmse:159.529	validation_1-rmse:394.289
[1400]	validation_0-rmse:151.362	validation_1-rmse:379.016
[1500]	validation_0-rmse:145.459	validation_1-rmse:370.436
[1600]	validation_0-rmse:140.447	validation_1-rmse:364.305
[1700]	validation_0-rmse:137.036	validation_1-rmse:360.287
[1800]	validation_0-rmse:134.67	validation_1-rmse:358.443
[1900]	validation_0-rmse:132.349	validation_1-rmse:355.445
[2000]	validation_0-rmse:130.315	validation_1-rmse:354.228
[2100]	validation_0-rmse:128.546	validation_1-rmse:353.346
[2200]	validation_0-rmse:126.654	validation_1-rmse:351.664
[2300]	validation_0-rmse:125.021	validation_1-rmse:350.541
[2400]	validation_0-rmse:123.295	validation_1-rmse:350.615
Stopping. Best iteration:
[2336]	validation_0-rmse:124.224	validation_1-rmse:350.032

0.7488332931180752
--------------------------
(1320, 2)
---
(0, 18)

In [27]:
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
print(sub.head())
print(sub.tail())

       id  forecastVolum
31680   1          265.0
31681   2          366.0
31682   3          159.0
31683   4          271.0
31684   5          451.0
         id  forecastVolum
36955  5364           82.0
36956  5365           78.0
36957  5366           88.0
36958  5367          200.0
36959  5368           54.0


In [43]:
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
print(sub.head())
print(sub.tail())

       id  forecastVolum
31680   1          246.0
31681   2          309.0
31682   3          172.0
31683   4          283.0
31684   5          376.0
         id  forecastVolum
36955  5364           99.0
36956  5365           94.0
36957  5366           92.0
36958  5367          216.0
36959  5368           65.0


In [35]:
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
print(sub.head())
print(sub.tail())

       id  forecastVolum
31680   1          271.0
31681   2          360.0
31682   3          169.0
31683   4          283.0
31684   5          447.0
         id  forecastVolum
36955  5364           85.0
36956  5365           79.0
36957  5366           93.0
36958  5367          202.0
36959  5368           53.0


In [36]:
sub[['id','forecastVolum']].round().astype(int).to_csv('submit/xgb1001nolog.csv', index=False)

In [113]:
for month in [21,22,23,24]: 
    m_type = 'lgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear','regMonth'] + stat_feat
    cate_feat = ['adcode','bodyType','model']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
#     print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.mt==month), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.mt==month), 'label'      ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'pred_label'      ] = sub['forecastVolum'].values
    
    
    
    
    
# sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
# sub.columns = ['id','forecastVolum']
# sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.07it/s]


all_idx  : 9 20
train_idx: 9 16
valid_idx: 17 17
test_idx : 21 21
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 128712	valid_1's l2: 98700.9
[200]	training's l2: 41958.5	valid_1's l2: 38841.5
[300]	training's l2: 23711.2	valid_1's l2: 28937.2
[400]	training's l2: 17316.6	valid_1's l2: 26511.4
[500]	training's l2: 13789.8	valid_1's l2: 25838.3
[600]	training's l2: 11344.6	valid_1's l2: 25413.5
[700]	training's l2: 9716.53	valid_1's l2: 25005.3
[800]	training's l2: 8521.85	valid_1's l2: 24786.1
[900]	training's l2: 7601.14	valid_1's l2: 24786.9
[1000]	training's l2: 6807.82	valid_1's l2: 24710.5
[1100]	training's l2: 6154.66	valid_1's l2: 24588.3
[1200]	training's l2: 5603.01	valid_1's l2: 24436.2
[1300]	training's l2: 5128.15	valid_1's l2: 24299.6
[1400]	training's l2: 4756.47	valid_1's l2: 24098.7
[1500]	training's l2: 4411.44	valid_1's l2: 23997.3
[1600]	training's l2: 4081.94	valid_1's l2: 23915.1
[1700]	training's l2: 3801.56	valid_1's l2: 23785

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.20it/s]


all_idx  : 9 21
train_idx: 9 17
valid_idx: 18 18
test_idx : 22 22
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 124896	valid_1's l2: 101622
[200]	training's l2: 41221.5	valid_1's l2: 40668.4
[300]	training's l2: 23551.2	valid_1's l2: 30796.4
[400]	training's l2: 17247.5	valid_1's l2: 28629.1
[500]	training's l2: 13644.1	valid_1's l2: 28447.5
[600]	training's l2: 11311.3	valid_1's l2: 28214.8
[700]	training's l2: 9665.8	valid_1's l2: 28306.6
Early stopping, best iteration is:
[602]	training's l2: 11275.5	valid_1's l2: 28202.4
0.7483585609319627
valid mean: 547.3275192104005
true  mean: 523.3159090909091
test  mean: 648.8015280661291
(1320, 2)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.75it/s]


all_idx  : 9 22
train_idx: 9 18
valid_idx: 19 19
test_idx : 23 23
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 121129	valid_1's l2: 101400
[200]	training's l2: 40304.7	valid_1's l2: 38675.6
[300]	training's l2: 23218.8	valid_1's l2: 27340.3
[400]	training's l2: 17179.7	valid_1's l2: 24853.9
[500]	training's l2: 13713.1	valid_1's l2: 23959.2
[600]	training's l2: 11471.2	valid_1's l2: 23400.8
[700]	training's l2: 9962.11	valid_1's l2: 22931.2
[800]	training's l2: 8881.21	valid_1's l2: 22567.1
[900]	training's l2: 8007.93	valid_1's l2: 22383.6
[1000]	training's l2: 7299.37	valid_1's l2: 22289.3
[1100]	training's l2: 6668.06	valid_1's l2: 22270.3
[1200]	training's l2: 6149.12	valid_1's l2: 22150.4
Early stopping, best iteration is:
[1196]	training's l2: 6170.05	valid_1's l2: 22128.1
0.7468931536269028
valid mean: 560.7094271141314
true  mean: 552.2295454545455
test  mean: 734.8046071367794
(1320, 2)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.18it/s]


all_idx  : 9 23
train_idx: 9 19
valid_idx: 20 20
test_idx : 24 24
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 118772	valid_1's l2: 145395
[200]	training's l2: 39386.3	valid_1's l2: 51071.7
[300]	training's l2: 22711.1	valid_1's l2: 32006.2
[400]	training's l2: 17040	valid_1's l2: 27020.4
[500]	training's l2: 13774.6	valid_1's l2: 25125.4
[600]	training's l2: 11675.5	valid_1's l2: 24421
[700]	training's l2: 10208	valid_1's l2: 24129.7
[800]	training's l2: 9205.22	valid_1's l2: 23964.6
[900]	training's l2: 8388.5	valid_1's l2: 23799.9
[1000]	training's l2: 7727.56	valid_1's l2: 23768.7
Early stopping, best iteration is:
[989]	training's l2: 7801.96	valid_1's l2: 23734.8
0.7917019686244774
valid mean: 595.4352742803294
true  mean: 620.0212121212121
test  mean: 856.408786058386
(1320, 2)


In [68]:
for month in [21,22,23,24]: 
    m_type = 'lgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear','regMonth','popularity'] + stat_feat
    cate_feat = ['adcode','bodyType','model']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
#     print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values
#     data.loc[(data.mt==(month)), 'pred_label'      ] = sub['forecastVolum'].values
    
    
    
    
    
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
# sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.37it/s]


all_idx  : 13 24
train_idx: 13 20
valid_idx: 21 21
test_idx : 25 25
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 86584.9	valid_1's l2: 151719
[200]	training's l2: 27784.3	valid_1's l2: 71704.5
[300]	training's l2: 14902.5	valid_1's l2: 57361.3
[400]	training's l2: 10243.5	valid_1's l2: 52224.7
[500]	training's l2: 7869.38	valid_1's l2: 50304.7
[600]	training's l2: 6385.06	valid_1's l2: 49569.7
[700]	training's l2: 5380.65	valid_1's l2: 48911.9
[800]	training's l2: 4634.41	valid_1's l2: 48274
[900]	training's l2: 4071.47	valid_1's l2: 48026.2
[1000]	training's l2: 3625	valid_1's l2: 47641.9
[1100]	training's l2: 3253.2	valid_1's l2: 47425.4
[1200]	training's l2: 2928.61	valid_1's l2: 47143.3
[1300]	training's l2: 2653.04	valid_1's l2: 46870.8
[1400]	training's l2: 2414.07	valid_1's l2: 46669.9
[1500]	training's l2: 2207.68	valid_1's l2: 46487.6
[1600]	training's l2: 2028.06	valid_1's l2: 46336.7
[1700]	training's l2: 1876.96	valid_1's l2: 46208.4
[

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.41it/s]


all_idx  : 13 25
train_idx: 13 21
valid_idx: 22 22
test_idx : 26 26
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 90083.5	valid_1's l2: 118412
[200]	training's l2: 28475.8	valid_1's l2: 59906.1
[300]	training's l2: 15309.6	valid_1's l2: 52080.3
[400]	training's l2: 10785.7	valid_1's l2: 50404.3
[500]	training's l2: 8324.54	valid_1's l2: 49632.1
[600]	training's l2: 6828.3	valid_1's l2: 49046.4
[700]	training's l2: 5813.9	valid_1's l2: 48997.8
Early stopping, best iteration is:
[648]	training's l2: 6301.83	valid_1's l2: 48762.4
0.7352932759641437


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.40it/s]


all_idx  : 13 26
train_idx: 13 22
valid_idx: 23 23
test_idx : 27 27
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 93591.6	valid_1's l2: 135289
[200]	training's l2: 29582.9	valid_1's l2: 52574.8
[300]	training's l2: 15864.8	valid_1's l2: 39181.2
[400]	training's l2: 11154.7	valid_1's l2: 36023.5
[500]	training's l2: 8739.67	valid_1's l2: 35331.5
[600]	training's l2: 7173.39	valid_1's l2: 34526.5
[700]	training's l2: 6126.06	valid_1's l2: 34402.5
[800]	training's l2: 5347.97	valid_1's l2: 34349
[900]	training's l2: 4770.82	valid_1's l2: 34426
Early stopping, best iteration is:
[806]	training's l2: 5308.31	valid_1's l2: 34325.1
0.7811424903161602


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.38it/s]


all_idx  : 13 27
train_idx: 13 23
valid_idx: 24 24
test_idx : 28 28
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 96023.9	valid_1's l2: 552065
[200]	training's l2: 29709.3	valid_1's l2: 354551
[300]	training's l2: 16067.9	valid_1's l2: 299015
[400]	training's l2: 11490.5	valid_1's l2: 282547
[500]	training's l2: 9067.98	valid_1's l2: 274322
[600]	training's l2: 7522.51	valid_1's l2: 271372
[700]	training's l2: 6477.28	valid_1's l2: 271395
Early stopping, best iteration is:
[658]	training's l2: 6879.88	valid_1's l2: 270441
0.6226761089064834


In [69]:
sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales0914.csv', index=False)

In [38]:
for month in [25,26,27,28]: 
    m_type = 'lgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear'] + stat_feat
    cate_feat = ['adcode','bodyType','model','regMonth']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
    print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
# sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.38s/it]


53 53
all_idx  : 13 24
train_idx: 13 20
valid_idx: 21 21
test_idx : 25 25
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 7291.26	valid_1's l2: 37932.3
[200]	training's l2: 3715.93	valid_1's l2: 35716.9
[300]	training's l2: 2515.27	valid_1's l2: 35194.8
[400]	training's l2: 1831.38	valid_1's l2: 34788.8
[500]	training's l2: 1397.64	valid_1's l2: 34608.6
[600]	training's l2: 1065.51	valid_1's l2: 34527.1
[700]	training's l2: 849.095	valid_1's l2: 34415.2
[800]	training's l2: 679.238	valid_1's l2: 34333.1
[900]	training's l2: 545.247	valid_1's l2: 34326.4
Early stopping, best iteration is:
[838]	training's l2: 624.086	valid_1's l2: 34316.6
0.7559303097480222
valid mean: 602.8972278482273
true  mean: 649.3121212121212
test  mean: 493.9801201875369


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.35s/it]


53 53
all_idx  : 13 25
train_idx: 13 21
valid_idx: 22 22
test_idx : 26 26
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 7936.91	valid_1's l2: 41558.5
Early stopping, best iteration is:
[75]	training's l2: 10761.5	valid_1's l2: 41117.3
0.7414955122487169
valid mean: 623.4500376146684
true  mean: 616.5537878787878
test  mean: 324.6451813135614


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.36s/it]


53 53
all_idx  : 13 26
train_idx: 13 22
valid_idx: 23 23
test_idx : 27 27
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 8653.84	valid_1's l2: 32560.4
[200]	training's l2: 4627.91	valid_1's l2: 32133.2
Early stopping, best iteration is:
[155]	training's l2: 5757.15	valid_1's l2: 31853.4
0.7794578141087838
valid mean: 639.6227869473659
true  mean: 673.0143939393939
test  mean: 479.55100257280156


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.40s/it]


53 53
all_idx  : 13 27
train_idx: 13 23
valid_idx: 24 24
test_idx : 28 28
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 9306.24	valid_1's l2: 302435
[200]	training's l2: 5107.21	valid_1's l2: 292129
[300]	training's l2: 3583.18	valid_1's l2: 289373
[400]	training's l2: 2709.06	valid_1's l2: 288389
[500]	training's l2: 2162.77	valid_1's l2: 287983
[600]	training's l2: 1746.75	valid_1's l2: 287504
[700]	training's l2: 1446.22	valid_1's l2: 287029
[800]	training's l2: 1196.94	valid_1's l2: 286429
[900]	training's l2: 1008.26	valid_1's l2: 286030
Early stopping, best iteration is:
[856]	training's l2: 1089.91	valid_1's l2: 286004
0.588139278724323
valid mean: 645.3917897243205
true  mean: 899.8204545454546
test  mean: 468.0356962498267
